# Tarea 2: Extracción de descriptores cromáticos de una imágen

## Instrucciones de uso
1. Asegúrate de tener la imagen **rice.png** en el mismo directorio del notebook o en la ruta '/content/rice.png'.  
2. El programa cargará la imagen, segmentará cada arroz y calculará los siguientes descriptores:  
   - Color promedio (toda la imagen)
   - Gradiente promedio por arroz
   - Contraste por arroz
   - Momentos de HU en escala de grises de cada grano de arroz

Al finalizar, se mostrará como resultado:  

- Un **DataFrame** impreso en pantalla con todos los descriptores cromáticos.  

**Cabe destacar que este código no realiza una segmentación precisa de cada grano de arroz. Debido a la presencia de ruido en la imagen, algunos granos podrían no ser detectados completamente y adicionalmente pueden incluirse píxeles que superen el umbral de binarización establecido en 150.**


In [ ]:
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt
import numpy.ma as ma
def mean_color(img):
  '''
  Calcula el promedio de intensidad en escala de grises de una imagen.
  '''

  c_promedio = np.mean(img)
  return c_promedio


def perimeter_opencv(region):
  '''
  Calcula el perímetro usando cv2.arcLength
  '''
  mask = region.image.astype("uint8")
  cnts, hc = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  cnt = cnts[0]
  return cv2.arcLength(cnt, True), cnt


def region_gradient(img, region):
  '''
  Estima el gradiente promedio a lo largo del perímetro de la región de interes.
  '''
  _, contours = perimeter_opencv(region)

  #generamos una matriz (fila/columna) con los contornos
  contornos = np.vstack(contours)

  #extraemos las coordenadas x,y locales
  x, y = zip(*contornos)
  x = np.array(x).reshape(-1, 1)
  y = np.array(y).reshape(-1, 1)

  #Coordenadas locales a globales
  minr, minc, maxr, maxc = region.bbox
  x = x + minc   # columnas
  y = y + minr   # filas

  #calculamos la derivada del perimetro
  dprom = perimeter_gradient(y, x, img)
  return dprom


def pixel_gradient(i, j, L):
  '''
  Calcula de gradiente para cada pixel a evaluar.
  '''
  Lx = -0.5 * L[i - 1, j] + 0.5 * L[i + 1, j]
  Ly = -0.5 * L[i, j - 1] + 0.5 * L[i, j + 1]
  return np.sqrt(Lx ** 2 + Ly ** 2)


def perimeter_gradient(I, J, im):
  '''
  Calcula el gradiente promedio a lo largo de una lista de puntos.
  '''
  n = len(I)
  out = 0
  for i in range(0, n):
      out = out + pixel_gradient(I[i], J[i], im)
  return (out / n).item()


def get_roi_bounds(region, shape):
  '''
  Devuelve los índices del ROI expandido alrededor de la región.
  '''
  minr, minc, maxr, maxc = region.bbox
  h, w = maxr - minr, maxc - minc
  H, W = shape

  iEmin = max(0, int(math.ceil(minr - h/2))) #Math.ceil para redondear hacia arriba
  jEmin = max(0, int(math.ceil(minc - w/2)))
  iEmax = min(H, iEmin + 2*h) # No se resta uno por el slice exclusivo que hace numpy.
  jEmax = min(W, jEmin + 2*w)



  return iEmin, iEmax, jEmin, jEmax

def calculate_contrast(imagen, objeto, r):
  '''
  Calcula el contraste K1, K2, K3 entre la región y su entorno recortado hasta la extension obtenida por la region de cada objeto.
  '''

  iEmin, iEmax, jEmin, jEmax = get_roi_bounds(r, imagen.shape) #Indices

  roi = (imagen[iEmin:iEmax, jEmin:jEmax]) # Se recorta la imagen con intensidades a los indices
  filtered_roi = ma.masked_equal(roi, 0)

  obj_roi = objeto[iEmin:iEmax, jEmin:jEmax] # Imagen binarizada

  region = filtered_roi[obj_roi>0] #Extraccion de la region con intensidades gracias a la imagen binaria
  entorno = filtered_roi[obj_roi<1] #Extraccion del entorno


  G_E = np.sum(entorno)/np.sum(obj_roi<1)
  G_R = np.sum(region)/np.sum(obj_roi>0)



  K_1 = ((G_R - G_E) / G_E)
  K_2 = ((G_R - G_E) / (G_R + G_E))
  K_3 = (np.log((G_R/G_E)))

  return K_1, K_2, K_3


In [88]:
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
from tabulate import tabulate


if __name__=='__main__':


  im = cv2.imread("/content/rice.png")
  gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  ret,bw = cv2.threshold(gray,150,255,cv2.THRESH_BINARY) # Binarizacion a 150


  etiquetas = label(bw)
  sts = regionprops(label_image=etiquetas, intensity_image=gray)

  indices = bw > 0

  data = []


  for region in sts:
      PosObject = np.zeros_like(bw)
      hu = region.moments_weighted_hu
      PosObject = np.logical_or(PosObject, etiquetas==region.label)
      ColorPromedioObjeto = mean_color(gray[PosObject])
      ColorPromedio = mean_color(gray)
      K1, K2, K3 = calculate_contrast(gray, PosObject, region)
      data.append({
          "Arroz": region.label,
          "Contraste (K1)": K1,
          "Contraste (K2)": K2,
          "Contraste (K3)": K3,
          "Gradiente Promedio": region_gradient(gray,region),
          "Hu1": hu[0],
          "Hu2": hu[1],
          "Hu3": hu[2],
          "Hu4": hu[3],
          "Hu5": hu[4],
          "Hu6": hu[5],
          "Hu7": hu[6],
          "Color Promedio por Objeto": ColorPromedioObjeto,


      })

  df = pd.DataFrame(data)

  print(tabulate(df, headers='keys', tablefmt='fancy_grid', showindex=False))


╒═════════╤══════════════════╤══════════════════╤══════════════════╤══════════════════════╤═════════════╤═════════════╤═════════════╤═════════════╤══════════════╤══════════════╤══════════════╤═════════════════════════════╕
│   Arroz │   Contraste (K1) │   Contraste (K2) │   Contraste (K3) │   Gradiente Promedio │         Hu1 │         Hu2 │         Hu3 │         Hu4 │          Hu5 │          Hu6 │          Hu7 │   Color Promedio por Objeto │
╞═════════╪══════════════════╪══════════════════╪══════════════════╪══════════════════════╪═════════════╪═════════════╪═════════════╪═════════════╪══════════════╪══════════════╪══════════════╪═════════════════════════════╡
│       1 │        0.6552    │       0.246761   │        0.503922  │            29.1477   │ 0.000965872 │ 1.40726e-07 │ 1.53653e-10 │ 7.84876e-12 │  1.1981e-22  │ -1.94171e-17 │  2.44822e-22 │                     185.957 │
├─────────┼──────────────────┼──────────────────┼──────────────────┼──────────────────────┼─────────────┼───